Explanation of this code

http://www.paulvangent.com/2016/04/01/emotion-recognition-with-python-opencv-and-a-face-dataset/

# Dataset: CK+, Jaffe, Rafd + Crawling Data

# 1. Organising the dataset

In [6]:
import os
os.getcwd()

'/Users/ryan/git_ryan/emotion_recognitoin'

In [7]:
from shutil import copyfile
import glob

emotions = ["neutral", "anger", "contempt", "disgust", "fear", "happy", "sadness", "surprise"] #Define emotion order
participants = glob.glob("source_emotion/*") #Returns a list of all folders with participant numbers

#Create folders

for x in participants:
    part = "%s" %x[-4:] #store current participant number
    for sessions in glob.glob("%s/*" %x): #Store list of sessions for current participant
        for files in glob.glob("%s/*" %sessions):
            current_session = files[20:-30]
            file = open(files, 'r')
            
            emotion = int(float(file.readline())) #emotions are encoded as a float, readline as float, then convert to integer.
            
            sourcefile_emotion = glob.glob("source_images/%s/%s/*" %(part, current_session))[-1] #get path for last image in sequence, which contains the emotion
            sourcefile_neutral = glob.glob("source_images/%s/%s/*" %(part, current_session))[0] #do same for neutral image
            
            dest_neut = "sorted_set/neutral/%s" %sourcefile_neutral[25:] #Generate path to put neutral image
            dest_emot = "sorted_set/%s/%s" %(emotions[emotion], sourcefile_emotion[25:]) #Do same for emotion containing image
            
            copyfile(sourcefile_neutral, dest_neut) #Copy file
            copyfile(sourcefile_emotion, dest_emot) #Copy file

# 2. Extracting faces

In [8]:
import glob
import cv2

faceDet = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
faceDet2 = cv2.CascadeClassifier("haarcascade_frontalface_alt2.xml")
faceDet3 = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
faceDet4 = cv2.CascadeClassifier("haarcascade_frontalface_alt_tree.xml")

emotions = ["neutral", "anger", "contempt", "disgust", "fear", "happy", "sadness", "surprise"] #Define emotions

In [9]:
files = glob.glob("sorted_set/%s/*" %emotion)
files

[]

In [10]:
def detect_faces(emotion):
    files = glob.glob("sorted_set/%s/*" %emotion) #Get list of all images with emotion
    filenumber = 0
    for f in files:
        frame = cv2.imread(f) #Open image
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #Convert image to grayscale
        
        #Detect face using 4 different classifiers
        face = faceDet.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
        face2 = faceDet2.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
        face3 = faceDet3.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
        face4 = faceDet4.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)

        #Go over detected faces, stop at first detected face, return empty if no face.
        if len(face) == 1:
            facefeatures = face
        elif len(face2) == 1:
            facefeatures == face2
        elif len(face3) == 1:
            facefeatures = face3
        elif len(face4) == 1:
            facefeatures = face4
        else:
            facefeatures = ""
        
        #Cut and save face
        for (x, y, w, h) in facefeatures: #get coordinates and size of rectangle containing face
            print "face found in file: %s" %f
            gray = gray[y:y+h, x:x+w] #Cut the frame to size
            
            try:
                out = cv2.resize(gray, (350, 350)) #Resize face so all images have same size
                cv2.imwrite("dataset/%s/%s.jpg" %(emotion, filenumber), out) #Write image
            except:
               pass #If error, pass file
        filenumber += 1 #Increment image number

for emotion in emotions: 
    detect_faces(emotion) #Call functiona

face found in file: sorted_set/neutral/00_002_00000001.png
face found in file: sorted_set/neutral/00_005_00000001.png
face found in file: sorted_set/neutral/00_006_00000001.png
face found in file: sorted_set/neutral/01_001_00000001.png
face found in file: sorted_set/neutral/01_002_00000001.png
face found in file: sorted_set/neutral/01_004_00000001.png
face found in file: sorted_set/neutral/01_006_00000001.png
face found in file: sorted_set/neutral/02_001_00000001.png
face found in file: sorted_set/neutral/02_002_00000001.png
face found in file: sorted_set/neutral/02_003_00000001.png
face found in file: sorted_set/neutral/02_004_00000001.png
face found in file: sorted_set/neutral/02_009_00000001.png
face found in file: sorted_set/neutral/03_001_00000001.png
face found in file: sorted_set/neutral/03_002_00000001.png
face found in file: sorted_set/neutral/03_006_00000001.png
face found in file: sorted_set/neutral/04_001_00000001.png
face found in file: sorted_set/neutral/04_002_00000001.p

# 3. Creating the training and classification set

In [90]:
import cv2
import glob
import random
import numpy as np

emotions = ["neutral", "anger", "contempt", "disgust", "fear", "happy", "sadness", "surprise"] #Emotion list
fishface = cv2.createFisherFaceRecognizer() #Initialize fisher face classifier

In [107]:
data = {}

def get_files(emotion): #Define function to get file list, randomly shuffle it and split 80/20
    files = glob.glob("dataset/%s/*" %emotion)
    random.shuffle(files)
    training = files[:int(len(files)*0.8)] #get first 80% of file list
    prediction = files[-int(len(files)*0.2):] #get last 20% of file list
    return training, prediction

def make_sets():
    training_data = []
    training_labels = []
    prediction_data = []
    prediction_labels = []
    for emotion in emotions:
        training, prediction = get_files(emotion)
        #Append data to training and prediction list, and generate labels 0-7
        for item in training:
            image = cv2.imread(item) #open image
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #convert to grayscale
            training_data.append(gray) #append image array to training data list
            training_labels.append(emotions.index(emotion))
    
        for item in prediction: #repeat above process for prediction set
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            prediction_data.append(gray)
            prediction_labels.append(emotions.index(emotion))

    return training_data, training_labels, prediction_data, prediction_labels

def run_recognizer():
    training_data, training_labels, prediction_data, prediction_labels = make_sets()
    
    print "training fisher face classifier"
    print "size of training set is:", len(training_labels), "images"
    fishface.train(training_data, np.asarray(training_labels))

    print "predicting classification set"
    cnt = 0
    correct = 0
    incorrect = 0
    for image in prediction_data:
        pred, conf = fishface.predict(image)
        if pred == prediction_labels[cnt]:
            correct += 1
            cnt += 1
        else:
            incorrect += 1
            cnt += 1
    return ((100*correct)/(correct + incorrect))

#Now run it
metascore = []
for i in range(0,10):
    correct = run_recognizer()
    print "got", correct, "percent correct!"
    metascore.append(correct)

print "\n\nend score:", np.mean(metascore), "percent correct!"

training fisher face classifier
size of training set is: 664 images
predicting classification set
got 26 percent correct!
training fisher face classifier
size of training set is: 664 images
predicting classification set
got 16 percent correct!
training fisher face classifier
size of training set is: 664 images
predicting classification set
got 20 percent correct!
training fisher face classifier
size of training set is: 664 images
predicting classification set
got 18 percent correct!
training fisher face classifier
size of training set is: 664 images
predicting classification set
got 26 percent correct!
training fisher face classifier
size of training set is: 664 images
predicting classification set
got 18 percent correct!
training fisher face classifier
size of training set is: 664 images
predicting classification set
got 22 percent correct!
training fisher face classifier
size of training set is: 664 images
predicting classification set
got 21 percent correct!
training fisher face cla

error: -------src-dir-------/opencv-2.4.10/modules/contrib/src/facerec.cpp:554: error: (-5) Empty training data was given. You'll need more than one sample to learn a model. in function train


In [88]:
data

{}

# 4. Optimizing Dataset

In [82]:
# For this I used google image search and the chrome plugin ZIG lite to batch-download the images from the results.

# 5. Looking at the difficulty

NameError: name 'training_data' is not defined

In [113]:
def run_recognizer():
    training_data, training_labels, prediction_data, prediction_labels = make_sets()
    
    print "training fisher face classifier"
    print "size of training set is:", len(training_labels), "images"
    fishface.train(training_data, np.asarray(training_labels))

    print "predicting classification set"
    cnt = 0
    correct = 0
    incorrect = 0
    for image in prediction_data:
        pred, conf = fishface.predict(image)
        if pred == prediction_labels[cnt]:
            correct += 1
            cnt += 1
        else:
            cv2.imwrite("difficult/%s_%s_%s.jpg" %(emotions[prediction_labels[cnt]], emotions[pred], cnt), image) #<-- this one is new
            incorrect += 1
            cnt += 1
    return ((100*correct)/(correct + incorrect))

In [115]:
run_recognizer()

training fisher face classifier
size of training set is: 86 images


error: -------src-dir-------/opencv-2.4.10/modules/contrib/src/lda.cpp:1010: error: (-5) At least two classes are needed to perform a LDA. Reason: Only one class was given! in function lda
